In [301]:
import numpy as np
import time
import copy
import random

In [251]:
layout = [0,3,1,3]

In [252]:
np_layout = np.array(layout)

In [253]:
np_layout

array([0, 3, 1, 3])

In [275]:
def count_attacking(board):
    N = len(board)
    count = 0
    attacking_queens = np.zeros(N)
    
    for col_1 in range(N - 1):
        row_1 = board[col_1]
    
        for j in range(1, N - col_1):
            col_2 = col_1 + j
            row_2 = board[col_2]

            if row_1 == row_2 or np.abs(row_1 - row_2) == np.abs(col_1 - col_2):
                attacking_queens[col_1] += 1
                attacking_queens[col_2] += 1
                count += 1

    return count, attacking_queens.astype(int)

In [276]:
attacking, queens = count_attacking(np_layout)
if attacking == 0:
    print(f'Found solution at {np_layout}')
else:
    print(f'{attacking = }')
    print(f'{queens = }')

attacking = 2
queens = array([1, 1, 0, 2])


In [298]:
test_np = np.array([1,2,0,5,5,5])

In [299]:
conflicted = np.where(test_np > 0)[0]

In [300]:
conflicted

array([0, 1, 3, 4, 5])

In [308]:
random.choice(conflicted)

np.int64(0)

In [309]:
conflicted.max()

np.int64(5)

In [7]:
def is_safe(board, col):
    row = board[col]
    for col_p in range(col):
        row_p = board[col_p]

        if row == row_p or np.abs(row - row_p) == np.abs(col - col_p):
            return False
    return True

In [8]:
safe = is_safe(np_layout, 3)
print(f'{safe}')

False


In [9]:
def n_queens_backtracking(N=8):

    board = np.empty(N)
    expanded_nodes = 0
    
    def backtrack(board,col):
        nonlocal expanded_nodes
        
        for row in range(N):
            expanded_nodes += 1
            board[col] = row
            
            if is_safe(board, col):
                if col + 1 < N:
                    if backtrack(board, col + 1):
                       return True 
                else:
                    return True

        return False

    return board.astype(int) if backtrack(board, 0 ) else None, expanded_nodes

In [11]:
N = 4
start = time.time()
solution, expanded_nodes = n_queens_backtracking(N)
print(f'''
{solution = }
{expanded_nodes = }
time = {time.time() - start :.5f}
''')


solution = array([1, 3, 0, 2])
expanded_nodes = 26
time = 0.00041



In [152]:
def update_domains(domains: dict, col, row):

    domains[col] = {row}
    N = len(domains)
    for c in range(1, N - col):

        domain_c = copy.deepcopy(domains[col + c]) 
        

        r = row
        if row in domain_c:
            domain_c.remove(row)
        r = row + c
        if r in domain_c:
            domain_c.remove(r)
        r = row - c
        if r in domain_c:
            domain_c.remove(r)

        
        if domain_c == set():
            return False

        domains[col + c] = domain_c 
        # print(domain_c)
    
    return domains

In [103]:
N = 8
domains_test = {}
domain_list = range(N)
domain_set = set(domain_list)
for i in domain_set:
    domains_test[i] = domain_set

In [168]:
r = {1,2,3}

In [ ]:
r.

In [149]:
r.remove(3)

In [150]:
r

set()

In [151]:
r == set()

True

In [ ]:
domains_test

In [88]:
domain_tested = update_domains(copy.deepcopy(domains_test), 0 , 0)

{2, 3}
{1, 3}
{1, 2}


In [86]:
domain_tested

{0: {0}, 1: {2, 3}, 2: {1, 3}, 3: {1, 2}}

In [184]:
def domains_prune(domains, col, row, removed):

    N = len(domains)
    for c in range(col + 1, N):

        dist = c - col

        for r in (row, row + dist, row - dist):
            if r in domains[c]:
                domains[c].remove(r)
                removed.append((c,r))

        if not domains[c]:
            return False
    
    return True

In [185]:
def restore_domains(domains, removed):
    for col, row, in removed:
        domains[col].add(row)

In [204]:
def n_queens_backtracking_with_forwardchecking(N=8):

    board = np.full(N,-1)
    expanded_nodes = 0
    domains = {col : set(range(N)) for col in range(N)}

    def backtrack(col):
        nonlocal expanded_nodes
        
        if col == N:
            return True
            
        for row in list(domains[col]):

            expanded_nodes += 1
            board[col] = row

            original_domain = domains[col]
            domains[col] = {row}
            
            removed = []
            if domains_prune(domains, col, row, removed):
                if backtrack(col + 1):
                    return True
                            
            restore_domains(domains, removed)
            domains[col] = original_domain
            board[col] = -1
        
        return False

    solved = backtrack(0)

    return board.astype(int) if solved else None, expanded_nodes

In [208]:
N = 20
start = time.time()
solution, expanded_nodes = n_queens_backtracking_with_forwardchecking(N)
print(f'''
{solution = }
{expanded_nodes = }
time = {time.time() - start :.5f}
''')


solution = array([ 0,  2,  4,  1,  3, 12, 14, 11, 17, 19, 16,  8, 15, 18,  7,  9,  6,
       13,  5, 10])
expanded_nodes = 140091
time = 0.74780



In [207]:
N = 8
start = time.time()
solution, expanded_nodes = n_queens_backtracking(N)
print(f'''
{solution = }
{expanded_nodes = }
time = {time.time() - start :.5f}
''')


solution = array([0, 4, 7, 5, 2, 6, 1, 3])
expanded_nodes = 876
time = 0.00228



In [209]:
l = [1,2,3,4,5]

In [210]:
n_l = np.array(l)

In [216]:
np.random.permutation(n_l)

array([2, 3, 5, 4, 1])

In [350]:
def n_queens_min_conflicts(N, max_depth=1000, max_restarts = 100):

    expanded_nodes = 0
    
    for restart in range(max_restarts):
        board = []
        for i in range(N):
            board.append(np.random.randint(N))
    
        board = np.array(board)
    
        for i in range(max_depth):
    
            count, queens = count_attacking(board)
            if count == 0:
                return board, expanded_nodes
    
            conflicted_queens = np.where(queens >0)[0]
    
            col = random.choice(conflicted_queens)
            
            heuristics = np.full(N, -1)
            original_row = board[col]
            for row in range(N):
                expanded_nodes += 1
                
                board[col] = row
                count, _ = count_attacking(board)
                heuristics[row] = count

            board[col] = original_row
            min_conflicts = heuristics.min()
            best_rows = np.where(heuristics == min_conflicts)[0]
    
            row = random.choice(best_rows)
    
            board[col] = row

            # print(f'{restart = }')
            # print(f'Iteration : {i}')
            # print(board)
    

    return False, expanded_nodes

In [370]:
N = 100
start = time.time()
solution, expanded_nodes = n_queens_min_conflicts(N)
print(f'''
{solution = }
{expanded_nodes = }
time = {time.time() - start :.5f}
''')


solution = array([29, 41, 92, 59, 62, 21, 69, 67, 38, 24, 90,  9, 65, 96, 50,  7, 39,
        4, 49, 66, 40, 83, 64, 42, 95, 74, 54, 99, 61, 43, 33,  3, 80, 84,
       88,  6, 75, 55, 70, 13, 28, 93, 19, 10,  1, 26, 89, 31,  0, 76, 71,
       36,  2, 23, 98, 51, 14, 58, 52, 97, 35, 22, 45, 16, 63, 73, 91, 72,
        5, 78, 32, 47, 94, 77, 17, 57,  8, 11, 46, 85, 25, 60, 79, 37, 12,
       18, 30, 87, 48, 82, 53, 86, 81, 20, 34, 68, 27, 56, 44, 15])
expanded_nodes = 10800
time = 32.23629



In [371]:
count_attacking(solution)

(0,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [230]:
l = []
for i in range(8):
    l.append(np.random.randint(8))

print(l)

[0, 4, 6, 7, 2, 5, 7, 5]


In [223]:
l

[6, 0, 2, 6, 2, 4, 7, 4]